

2. ***Naive-Bayes***


In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

**Mounting G drive**


> You need to authenticate to GD in order to get the AWID dataset with the scripts.



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Get the resources**

> Get all necessary resources from Google Drive, this works after the authentification.these resources are compressed to save some time.


In [0]:
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Trn_with_headers.zip" .
!cp -rn "/content/drive/My Drive/AWID/AWID-CLS-R-Tst_with_headers.zip" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/utils.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/main.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/imports.py" .
!cp -rn "/content/drive/My Drive/Mahamdi's PFE/for Collab/class_report.py" .

!unzip AWID-CLS-R-Trn_with_headers.zip 
!unzip AWID-CLS-R-Tst_with_headers.zip 
!rm *.zip
!rm -rf sample_data/

Archive:  AWID-CLS-R-Trn_with_headers.zip
  inflating: AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv  
Archive:  AWID-CLS-R-Tst_with_headers.zip
  inflating: AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv  


***Imports***


> Import all necessary libraries.



In [0]:
from imports import *

***Read the data and replace the '?' string in the Pandas DataFrame with a NumPy NaN value***

In [0]:
train_path = '/content/AWID-CLS-R-Trn/AWID-CLS-R-Trn_with_headers.csv'
test_path = '/content/AWID-CLS-R-Tst/AWID-CLS-R-Tst_with_headers.csv'
train_df_raw = pd.read_csv(train_path, low_memory=False)
test_df_raw = pd.read_csv(test_path, low_memory=False)
train_df_raw.replace('?', np.nan, inplace=True)
test_df_raw.replace('?', np.nan, inplace=True)


**Drop these colomns**


> Each one of the dropped columns has a Nan rate of 1.
> Note that we need to drop them from both of training and validation sets.


In [0]:
del test_df_raw['frame.dlt']
del test_df_raw['wlan.qos.buf_state_indicated']
del test_df_raw['radiotap.mactime']
del test_df_raw['wlan.ba.bm']
del test_df_raw['wlan.ba.control.ackpolicy']
del test_df_raw['wlan.ba.control.cbitmap']
del test_df_raw['wlan.ba.control.multitid']
del test_df_raw['wlan.bar.compressed.tidinfo']
del test_df_raw['wlan.bar.type']
del test_df_raw['wlan_mgt.fixed.fragment']
del test_df_raw['wlan_mgt.fixed.sequence']
del test_df_raw['wlan_mgt.tcprep.link_mrg']
del test_df_raw['wlan_mgt.tcprep.trsmt_pow']


del train_df_raw['frame.dlt']
del train_df_raw['wlan.qos.buf_state_indicated']
del train_df_raw['radiotap.mactime']
del train_df_raw['wlan.ba.bm']
del train_df_raw['wlan.ba.control.ackpolicy']
del train_df_raw['wlan.ba.control.cbitmap']
del train_df_raw['wlan.ba.control.multitid']
del train_df_raw['wlan.bar.compressed.tidinfo']
del train_df_raw['wlan.bar.type']
del train_df_raw['wlan_mgt.fixed.fragment']
del train_df_raw['wlan_mgt.fixed.sequence']
del train_df_raw['wlan_mgt.tcprep.link_mrg']
del train_df_raw['wlan_mgt.tcprep.trsmt_pow']


***Data pre-processing***


> We need to treat the data before passing it to our model.



***Change any columns of strings in a panda's dataframe to a column of categorical values.***
 

> This applies the changes inplace.




In [0]:
str_to_cat(train_df_raw)

***Changes any columns of strings in df into categorical variables using trn as a template for the category codes***
 

> This applies the changes inplace too.




In [0]:
apply_cats(test_df_raw,train_df_raw)

***We're still not quite done !***
 
> For instance we have lots of missing values, which we can't pass directly to the model.




***We call  the function proc_df() to process a pandas dataframe !***
 

*   *proc_df*  takes a data frame df and splits off the response variable in our case the *class* colomn, and changes the df into an entirely numeric dataframe. For each column of df which is not in skip_flds nor in ignore_flds, na values are replaced by the median value of the column.
*    Three other funtions are called inside this function:
    *    ***numericalize()*** : Changes the column col from a categorical type to it's integer codes.
    *    ***fix_missing()*** : Fill missing data in a column of df with the median, and add a {name}_na column 
    which specifies if the data was missing.
    *    ***scale_vars()*** : Scale tha data.



In [0]:
x_train, y_train , nas , mapper = proc_df(train_df_raw, 'class', do_scale=True)
x_valid, y_valid , nas , _ = proc_df(test_df_raw,'class', na_dict=nas, do_scale=True, mapper=mapper)

***Classification with Naive-Bayes***

Instantiate and train the model

In [0]:
gnb = GaussianNB() 
%time gnb.fit(x_train, y_train) 

CPU times: user 4.87 s, sys: 109 ms, total: 4.98 s
Wall time: 4.98 s


GaussianNB(priors=None, var_smoothing=1e-09)

Evaluate the model:

In [0]:
# making predictions on the testing set 
predict = gnb.predict(x_valid)
CM = confusion_matrix(y_valid, predict)
CR = classification_report (y_valid,predict)
report_with_auc = class_report(y_true=y_valid, y_pred=predict,y_score=gnb.predict_proba(x_valid))
print("************************the original set *************************")
print("=== Confusion Matrix ===\n",CM)
print("\n=== Report with classification report  ===\n\n",CR)
print("\n=== Report with auc  ===\n\n",report_with_auc)

************************the original set *************************
=== Confusion Matrix ===
 [[  8097      0      0      0]
 [  1473      0      0  18606]
 [   360      0  13655   2667]
 [249649  15340      0 265795]]

=== Report with classification report  ===

               precision    recall  f1-score   support

           0       0.03      1.00      0.06      8097
           1       0.00      0.00      0.00     20079
           2       1.00      0.82      0.90     16682
           3       0.93      0.50      0.65    530784

    accuracy                           0.50    575642
   macro avg       0.49      0.58      0.40    575642
weighted avg       0.88      0.50      0.63    575642


=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.031193  1.000000  0.060499    8097.0  259579.0  0.885244
1             0.000000  0.000000  0.000000   20079.0   15340.0  0.776601
2             1.000000  0.818547  0.900221   16682.0  

***features selection***
> We are going to test our model in different sets.

> Want to know more about these sets? The details are in the report.





In [0]:
features2 = ['radiotap.datarate', 'wlan.fc.type_subtype', 'wlan.seq','wlan_mgt.fixed.capabilities.preamble','wlan_mgt.fixed.timestamp','wlan_mgt.fixed.beacon','wlan_mgt.tim.dtim_period','data.len','class']
features3 = ['frame.time_epoch' , 'frame.time_relative' , 'radiotap.datarate' ,'wlan.fc.type_subtype' ,'wlan.fc.ds' ,'wlan.fc.retry' ,'wlan.fc.protected' ,'wlan.ta' ,'wlan.seq' ,'wlan_mgt.fixed.capabilities.ess' ,'wlan_mgt.fixed.capabilities.preamble' ,'wlan_mgt.fixed.capabilities.short_slot_time' ,'wlan_mgt.fixed.timestamp' ,'wlan_mgt.fixed.beacon' ,'wlan_mgt.fixed.auth.alg' ,'wlan_mgt.fixed.auth_seq' ,'wlan_mgt.tim.dtim_period' ,'wlan_mgt.rsn.akms.type' ,'wlan.wep.key' ,'data.len' ,'class']
features4 = ['frame.time_epoch','frame.time_delta','frame.time_delta_displayed','frame.time_relative','frame.len','frame.cap_len','wlan.fc.subtype','wlan.fc.ds','wlan.duration','wlan.ra','wlan.da','wlan.ta','wlan.sa','wlan.seq','wlan.wep.iv','wlan.wep.icv','data.len','class']

# set 2
train2 = train_df_raw[features2]
test2 = test_df_raw[features2]  
str_to_cat(train2)
apply_cats(test2,train2)
x_valid2 , y_valid2 , nas2 , mapper2 = proc_df(test2, 'class' , do_scale = True)
x_train2 , y_train2 , nas2 , _ = proc_df(train2, 'class',na_dict=nas2 , do_scale = True , mapper = mapper2)

# set 3
train3 = train_df_raw[features3]
test3 = test_df_raw[features3] 
str_to_cat(train3)
apply_cats(test3,train3)
x3,y3,nas3 = proc_df(train3, 'class')
x_valid3 , y_valid3 , nas3 , mapper3 = proc_df(test3,'class' , do_scale = True)
x_train3 , y_train3 , nas3 , _ = proc_df(train3, 'class',na_dict=nas3 , do_scale = True , mapper = mapper3)

# set 4
train4 = train_df_raw[features4]
test4 = test_df_raw[features4] 
str_to_cat(train4)
apply_cats(test4,train4)
x4,y4,nas4 = proc_df(train4, 'class')
x_valid4 , y_valid4 , nas4 , mapper4 = proc_df(test4,'class' , do_scale = True)
x_train4 , y_train4 , nas4 , _ = proc_df(train4, 'class',na_dict=nas4 , do_scale = True , mapper = mapper4)



/content/utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if is_string_dtype(c): df[n] = c.astype('category').cat.as_ordered()
/content/utils.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[n] = c.astype('category').cat.as_ordered()
/content/utils.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_g

In [0]:
# set2
gnb2 = GaussianNB() 
%time gnb2.fit(x_train2, y_train2) 
  
# making predictions on the testing set 
predict2 = gnb2.predict(x_valid2)
CM2 = confusion_matrix(y_valid2, predict2)
CR2 = classification_report (y_valid2,predict2)
report_with_auc2 = class_report(y_true=y_valid2, y_pred=predict2,y_score=gnb2.predict_proba(x_valid2))
print("************************set2 *************************")
print("=== Confusion Matrix ===\n",CM2)
print("\n=== Report with auc  ===\n\n",report_with_auc2)

CPU times: user 376 ms, sys: 2.97 ms, total: 379 ms
Wall time: 379 ms
************************set2 *************************
=== Confusion Matrix ===
 [[  7994    103      0      0]
 [  1473      0  18606      0]
 [   156   1859  14333    334]
 [230586 182733  16155 101310]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.033279  0.987279  0.064388    8097.0  240209.0  0.930414
1             0.000000  0.000000  0.000000   20079.0  184695.0  0.469457
2             0.291950  0.859190  0.435812   16682.0   49094.0  0.933060
3             0.996714  0.190869  0.320384  530784.0  101644.0  0.890157
avg / total   0.927972  0.214781  0.308953  575642.0  575642.0  0.732893


In [0]:
# set3
gnb3 = GaussianNB() 
%time gnb3.fit(x_train3, y_train3) 
  
# making predictions on the testing set 
predict3 = gnb3.predict(x_valid3)
CM3 = confusion_matrix(y_valid3, predict3)
CR3 = classification_report (y_valid3,predict3)
report_with_auc3 = class_report(y_true=y_valid3, y_pred=predict3,y_score=gnb3.predict_proba(x_valid3))
print("************************set3 *************************")
print("=== Confusion Matrix ===\n",CM3)
print("\n=== Report with auc  ===\n\n",report_with_auc3)

CPU times: user 724 ms, sys: 10.9 ms, total: 735 ms
Wall time: 735 ms


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


************************set3 *************************
=== Confusion Matrix ===
 [[  8091      0      0      6]
 [  1473      0      0  18606]
 [   181      0  13730   2771]
 [217014      0    198 313572]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.035681  0.999259  0.068902    8097.0  226759.0  0.942388
2             0.985784  0.823043  0.897092   16682.0   13928.0  0.270903
3             0.936162  0.590771  0.724403  530784.0  334955.0  0.193700
avg / total   0.892279  0.582642  0.694919  555563.0  555563.0  0.925943


In [0]:
# set4
gnb4 = GaussianNB() 
%time gnb4.fit(x_train4, y_train4) 
  
# making predictions on the testing set 
predict4 = gnb4.predict(x_valid4)
CM4 = confusion_matrix(y_valid4, predict4)
CR4 = classification_report (y_valid4,predict4)
report_with_auc4 = class_report(y_true=y_valid4, y_pred=predict4,y_score=gnb4.predict_proba(x_valid4))
print("************************set4 *************************")
print("=== Confusion Matrix ===\n",CM4)
print("\n=== Report with auc  ===\n\n",report_with_auc4)

CPU times: user 589 ms, sys: 5.85 ms, total: 595 ms
Wall time: 595 ms
************************set4 *************************
=== Confusion Matrix ===
 [[  8097      0      0      0]
 [  1473      0      0  18606]
 [   360      0  13644   2678]
 [253506  13504      0 263774]]

=== Report with auc  ===

              precision    recall  f1-score   support      pred       AUC
0             0.030736  1.000000  0.059639    8097.0  263436.0  0.894779
1             0.000000  0.000000  0.000000   20079.0   13504.0  0.767519
2             1.000000  0.817888  0.899822   16682.0   13644.0  0.969005
3             0.925334  0.496952  0.646630  530784.0  285058.0  0.591417
avg / total   0.882638  0.495994  0.623156  575642.0  575642.0  0.904133


***Save the models***

> With the ***dill*** framework,which is a utility for serialization of python objects.



In [0]:
os.makedirs("NB_Models", exist_ok=True)

dill.dump(gnb, open("NB_Models/nb.obj", "wb")) 
dill.dump(gnb2, open("NB_Models/nb2.obj", "wb"))
dill.dump(gnb3, open("NB_Models/nb3.obj", "wb"))
dill.dump(gnb4, open("NB_Models/nb4.obj", "wb"))

In [0]:
!zip -r NB_Models.zip NB_Models
!cp -f NB_Models.zip "/content/drive/My Drive/Mahamdi's PFE/Models"

  adding: NB_Models/ (stored 0%)
  adding: NB_Models/nb1.obj (deflated 20%)
  adding: NB_Models/nb2.obj (deflated 61%)
  adding: NB_Models/nb3.obj (deflated 22%)
  adding: NB_Models/nb4.obj (deflated 24%)
